In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
# Load CSV file
df = pd.read_csv("salesmonthly.csv")

# Show first few rows
df.head()


In [ ]:
df['datum'] = pd.to_datetime(df['datum'])  # Convert 'datum' to datetime
df.set_index('datum', inplace=True)       # Set date column as index
df.info()


In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(14, 6))
df.plot(title="Monthly Sales Trends per Drug Category (2014–2019)", grid=True)
plt.xlabel("Date")
plt.ylabel("Sales")
plt.tight_layout()
plt.show()

In [ ]:
!pip install fbprophet --upgrade

In [ ]:
%pip install Prophet

In [ ]:
from prophet import Prophet

In [ ]:
# Prepare the DataFrame
df_prophet = df[['N02BE']].reset_index()
df_prophet.columns = ['ds', 'y']
df_prophet.head()

In [ ]:
model = Prophet()
model.fit(df_prophet)

In [ ]:
future = model.make_future_dataframe(periods=12, freq='M')
future.tail()

In [ ]:
forecast = model.predict(future)

# Plot forecast
fig1 = model.plot(forecast)
plt.title("N02BE Drug Forecast (12 Months Ahead)")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.grid(True)
plt.show()

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(12)

In [ ]:
from prophet.make_holidays import make_holidays_df

# Add Danish holidays (DK available)
model = Prophet(weekly_seasonality=True, yearly_seasonality=True)
model.add_country_holidays(country_name='DK')

# Refit the model
model.fit(df_prophet)

# Forecast again
future = model.make_future_dataframe(periods=12, freq='M')
forecast = model.predict(future)

# Plot updated forecast
model.plot(forecast)
plt.title("N02BE Forecast with Seasonality + Danish Holidays")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
%pip install statsmodels

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

# Reuse df with datetime index and 'N02BE' column
ts = df['N02BE']

# Train/test split
train_arima = ts[:-12]
test_arima = ts[-12:]

# Fit ARIMA model (manual order or auto_arima later)
model_arima = ARIMA(train_arima, order=(1,1,1))
model_fit = model_arima.fit()

# Forecast
forecast_arima = model_fit.forecast(steps=12)

# Plot
plt.figure(figsize=(12, 5))
plt.plot(test_arima.index, test_arima, label="Actual")
plt.plot(test_arima.index, forecast_arima, label="ARIMA Forecast")
plt.legend()
plt.title("N02BE Forecast – ARIMA vs Actual")
plt.grid(True)
plt.show()

In [ ]:
import joblib
joblib.dump(model, 'prophet_n02be_model.pkl')